# Урок 2. Метрики качества классификации ч.1

В уроке мы разбирали матрицу ошибок и некоторые метрики, основанные на терминах матрицы ошибок. Здесь же посмотрим, как эти метрики применять на практике и что  по ним можно выяснить.

### 1.1
Выкачайте тренировочный датасет Titanic - train.csv - с сайта [kaggle](https://www.kaggle.com/c/titanic/data). С помощью функции pd.read_csv() загрузите данные в датафрейм. Выведите первые 20 строк и проанализируйте данные: какие колонки присутствуют (более конкретная информация по ним есть на сайте kaggle), каким образом в них обозначены данные и какие типы данных используются (используйте pandas.dtypes). 

In [1]:
import pandas as pd

In [2]:
titanic = pd.read_csv('train.csv')
titanic.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


В данном датасете присутствуют следующие названия колонок:
1. PassengerId: ID пассажира - числовое целое
2. Survived: выжил пассажир, или нет. Поле числовое
3. Pclass: класс пассажира - числовое, категориальное
4. Name: имя пассажира - текстовое
5. Sex: пол пассажира - текстовое
6. Age: возраст пассажира - числовое с плавающей точкой
7. Parch: количество родителей/детей на борту - числовое целое
8. Ticket: номер билета - текстовое
9. Fare: стоимость билета - числовое с плавающей точкой
10. Cabin: номер каюты - текстовое
11. Embarked: порт посадки - текстовое

In [4]:
titanic[['Age', 'Pclass', 'SibSp', 'Parch', 'Fare']].describe()

,Age,Pclass,SibSp,Parch,Fare
count,714.000000,891.000000,891.000000,891.000000,891.000000
mean,29.699118,2.308642,0.523008,0.381594,32.204208
std,14.526497,0.836071,1.102743,0.806057,49.693429
min,0.420000,1.000000,0.000000,0.000000,0.000000
25%,20.125000,2.000000,0.000000,0.000000,7.910400
50%,28.000000,3.000000,0.000000,0.000000,14.454200
75%,38.000000,3.000000,1.000000,0.000000,31.000000
max,80.000000,3.000000,8.000000,6.000000,512.329200


### 1.2
Проверьте, имеются ли пропущенные значения в колонках, и выведите сумму всех пропущенных значений в каждой из колонок.

In [5]:
titanic.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

В колонках пропущенны следущее количество данных:
1. Age - кол-во пропущенных значений-177
2. Cabin - кол-во пропущенных значений-687
3. Embarked - кол-во пропущенных значений - 2

### 1.3
Замените все пропущенные значения колонки Age на медианы в зависимости от пола человека: т.е. если пол человека в строке с пропущенным значением "male", заменяете пропущенное значение возраста на медиану по всем известным возрастам мужчин, и наоборот. Выведите медианы возраста в зависимости от пола. Пропущенные значения колонок Cabin и Embarked замените на U (Unknown).

Примечание: для группировки по полу можно использовать метод df.groupby(), который имеет встроенные агрегатные функции (в т.ч. для вычисления медианы).  Для установки пропущенных значений в датасете воспользуйтесь функцией df.apply().

In [6]:
#Заполнил пропущенные значения в колонке Age 
sex_median = titanic.groupby('Sex')['Age'].median()
print(f"Медианое значение возрастов от пола:\nмужчины - {sex_median['male']}, женщины - {sex_median['female']}")
titanic.loc[(titanic.Sex=='male') & (titanic['Age'].isna()), 'Age'] = sex_median['male']
titanic.loc[(titanic.Sex=='female') & (titanic['Age'].isna()), 'Age'] = sex_median['female']


Медианое значение возрастов от пола:
мужчины - 29.0, женщины - 27.0


In [7]:
#Заполнил пропущенные значения в колонке номеров кают и портов посадки
titanic.loc[titanic['Cabin'].isna(), 'Cabin'] ='U'
titanic.loc[titanic['Embarked'].isna(), 'Embarked'] = titanic.loc[titanic['Embarked'].isna(), 'Embarked'].apply(lambda data: 'U')

In [8]:
titanic.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

### 1.4
Выведите возраст пассажиров с PassengerID = [6, 20]. Убедитесь, что заполнены все пропущенные значения (воспользуйтесь функцией df.isnull() ).

In [9]:
titanic_age_6_20 = titanic[(titanic['PassengerId'] >= 6) & (titanic['PassengerId'] <= 20)]['Age']
print(f'Возраст людей с ID от 6 до 20 включительно: \n{titanic_age_6_20.values}')
print('Количество пропущенных значений:', titanic_age_6_20.isnull().sum() )



Возраст людей с ID от 6 до 20 включительно: 
[29. 54.  2. 27. 14.  4. 58. 20. 39. 14. 55.  2. 29. 31. 27.]
Количество пропущенных значений: 0


Так как не все классификаторы могут работать с категориальными признаками, в данном случае необходимо заменить их на числовые.

Также можно было бы выделить расширенные признаки, такие как статус пассажира (Mr., Mrs., Miss., Dr., Master. и т.д.), собственный индекс билета, индекс палубы и другие, но здесь ограничимся выбором более простых признаков.

### 1.5
В колонке Sex замените значения на 0, если пол "male", и на 1, если "female". В колонке Embarked замените параметры "U", "S", "C", "Q" на 0, 1, 2, 3 соответственно. Отбросьте колонки PassengerId, Name, Ticket, Cabin. Выведите первые 20 строк получившегося набора данных.

In [10]:
titanic['Sex'] = titanic['Sex'].apply(
    lambda sex: 0 if sex=='male' else (1 if sex=='female' else exec('raise(ValueError("No valid Sex name"))') )
 )

In [11]:
def Embarked_code(code):
    if code == 'U':
        return 0
    elif code == 'S':
        return 1
    elif code == 'C':
        return 2
    elif code == 'Q':
        return 3

titanic['Embarked'] = titanic['Embarked'].apply(Embarked_code)

In [12]:
del titanic['PassengerId']
del titanic['Name']
del titanic['Ticket']
del titanic['Cabin']
titanic.head(20)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,1
1,1,1,1,38.0,1,0,71.2833,2
2,1,3,1,26.0,0,0,7.9250,1
3,1,1,1,35.0,1,0,53.1000,1
4,0,3,0,35.0,0,0,8.0500,1
5,0,3,0,29.0,0,0,8.4583,3
6,0,1,0,54.0,0,0,51.8625,1
7,0,3,0,2.0,3,1,21.0750,1
8,1,3,1,27.0,0,2,11.1333,1
9,1,2,1,14.0,1,0,30.0708,2


### 1.6
Приступим к построению классификаторов. **Условимся, что, если функция или объект модели имеют параметр random_state, то устанавливаем его равным 17 в каждом из случаев.**

Разделите данные на тренировочный и тестовый датасеты, установив размер тестового как 0.25 (первая колонка Survived является целевой, поэтому необходимо сначала ее отделить от признаков).

In [13]:
from sklearn.model_selection import train_test_split 

In [14]:
x_train, x_test, y_train, y_test = train_test_split(titanic[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']],
                                                   titanic['Survived'], random_state=17)

### 1.7
Обучите на полученных выборках несколько классификаторов, которые импортированы ниже.

**Примечание: в методе ближайших соседей используйте количество соседей, равное 5.**

In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [16]:
kn = KNeighborsClassifier(n_neighbors=5).fit(x_train, y_train)
gnb = GaussianNB().fit(x_train, y_train)
dtc = DecisionTreeClassifier(random_state=17).fit(x_train, y_train)
ir = LogisticRegression(random_state=17).fit(x_train, y_train)


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Очевидно, в этом случае использовать метрику accuracy, которую мы применяли до этого, не совсем правильно: она не даст верную оценку классификатору, потому как выживших явно меньше, чем погибших.

Вспомним про такие метрики как точность и полнота. Здесь они подойдут хорошо, так как у нас имеются положительный и отрицательный классы, и определить корректно один из них может оказаться более важным .

### 1.8
В каждом из пунктов следующего теста выберите, что важнее максимизировать: точность (precision) или полноту (recall). Выпишите ответы.

1. Вероятность того, что при определенной поломке самолета он сможет долететь до пункта назначения (1 - долетел, 0 - не долетел).
2. Предсказание, представляет ли человек опасность, по анализу психического состояния (1 - представляет опасность, 0 - не представляет опасности).
3. Предсказание ухода клиента (1 - клиент ушел, 0 - остался).
4. Выявление рака на основе медицинских показателей (1 - болен раком, 0 - здоров).
5. Предсказание летальности при наблюдаемой мутации (1 - выживание, 0 - летальный исход).
6. Определение важности происшествия для экстренных служб (1 - важно, 0 - неважно).
7. Окупятся ли вложения в бизнес (1 - окупятся, 0 - не окупятся).

Ваш ответ:

1. <br> полноту (recall)
2. <br> полноту (recall)
3. <br> полноту (recall)
4. <br> полноту (recall)
5. <br> полноту (recall)
6. <br> полноту (recall)
7. <br> точность (precision)

### 1.9
Определите, что важнее использовать в случае датасета Titanic: полноту или точность. Объясните, почему.
<br>
<br> Ответ:
В данном случае я взял полноту (Recall), на мой взгляд, если оценивать вероятность гибели людей, нам безопаснее переоценить ее и учесть дополнительные риски, чем недооценить, и пропустить важные детали.
### 1.10
Для каждого классификатора выведите матрицу ошибок и самостоятельно рассчитайте метрику recall (расчеты должны присутствовать). Проверьте расчеты, воспользовавшись встроенной метрикой recall_score.

In [17]:
from sklearn.metrics import confusion_matrix, recall_score

In [29]:
models = (kn, gnb, dtc, ir)
for model in models:
    pred = model.predict(x_test)
    matrix = confusion_matrix(y_test, pred)
    recall = matrix[0][0]/(matrix[0][0] + matrix[1][0])
    print(model, matrix, f"recall = \t\t{recall}", sep='\n\n')
    print(f"recall_score = \t{recall_score(y_test, pred)}")
    print('-' * 50, '\n')

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

[[110  21]
 [ 47  45]]

recall = 		0.7006369426751592
recall_score = 	0.4891304347826087
-------------------------------------------------- 

GaussianNB(priors=None, var_smoothing=1e-09)

[[111  20]
 [ 27  65]]

recall = 		0.8043478260869565
recall_score = 	0.7065217391304348
-------------------------------------------------- 

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=17, splitter='best')

[[112  19]
 [ 26  66]]

recall = 		0.8115942028985508
recall_score = 	0.717391

### 1.11
Выберите ту модель, на которой метрика recall давала лучший ответ, и рассчитайте для нее precision, используя встроенную функцию.

In [26]:
from sklearn.metrics import precision_score

In [27]:
dtc_ps = recall_score(y_test, dtc_pred)
print('Precision score для DecisionTrees:', dtc_ps)



Precision score для DecisionTrees: 0.717391304347826
